### Importando pacotes necessarios

In [20]:
import openai
import pandas as pd

### Criando cliente OpenAI

In [21]:
client  =openai.Client()

### Carregando arquivo para análise

In [22]:
file = client.files.create (
    file=open("sales_data.csv","rb"),
    purpose = "assistants"
)

#### * Imprimindo id de registro do arquivo

In [23]:
print(file.id)

file-8riPWZe5gjZ8zf8yCXdpj3


### Criando Assistente

In [24]:
assistant = client.beta.assistants.create(
    name="Analista de dados",
    instructions = "Você é uma analista de dados, que tem a responsabilidade de analisar dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

### Carregando pergunta

In [25]:
# pergunta = "Qual o rating médio das vendas do supermecado?"
pergunta = "Gere um grafico de pizza com percentual de vendas por sexo"

### Criando Thread

In [26]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = pergunta
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\1131807865.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\1131807865.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


### Executando Thread

In [27]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id, 
    instructions="Premium"
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\1460489382.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


#### * Verificando status da Thread

In [30]:
import time

while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status in ["completed", "failed", "cancelled"]:
        break
    time.sleep(1)


C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\2160164208.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)


#### * Exibindo mensagens

In [31]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

for msg in messages.data:
    print(msg.role, ":", msg.content[0].text.value)


C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\1076570648.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


AttributeError: 'ImageFileContentBlock' object has no attribute 'text'

## Analise dos passos do modelo:


#### Carregando passos

In [ ]:

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\975397378.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


#### Imprimindo passos

In [34]:
for step in run_steps.data:
    print(f"Step: {step.step_details.type}")
    if step.type == "message_creation":
        print("Mensagem criada pelo modelo:")
        message = client.beta.threads.messages.retrieve(
            thread_id = thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type =="image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f'files/{file.id}.png',"wb") as f:
                f.write(image_data.read())
                print(f'Imagem {file_id}, salva!')

    elif step.type == "tool_calls":
        for call in step.step_details.tool_calls:
            print(call.code_interpreter.input)
            # print("Ferramenta chamada:", call.function.name if call.type == "function" else call.type)



Step: message_creation
Mensagem criada pelo modelo:


C:\Users\frpbo\AppData\Local\Temp\ipykernel_8468\3664164769.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Imagem file-BWQCWQkuyHgQN6NpowYukd, salva!
Step: tool_calls
import matplotlib.pyplot as plt

# Somar o total de vendas por gênero
sales_by_gender = data.groupby('Gender')['Total'].sum()

# Criar o gráfico de pizza
plt.figure(figsize=(8, 8))
plt.pie(sales_by_gender, labels=sales_by_gender.index, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff'])
plt.title('Percentual de Vendas por Sexo')
plt.show()
Step: message_creation
Mensagem criada pelo modelo:
O conjunto de dados contém informações sobre transações de vendas, incluindo colunas para "Gender" (sexo) e "Total" (valor total da venda).

Para gerar um gráfico de pizza que mostra o percentual de vendas por sexo, vamos somar as vendas totais para cada grupo de gênero e criar o gráfico com base nesses totais. Vamos fazer isso agora.
Step: tool_calls
import pandas as pd

# Carregar o arquivo para inspecionar os dados
file_path = '/mnt/data/file-8riPWZe5gjZ8zf8yCXdpj3'
data = pd.read_csv(file_path)

# Mostrar as primeiras linha